In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import splitfolders
splitfolders.ratio("Data", output="output",
    seed=1337, ratio=(.8, .2), group_prefix=None, move=False) # default values

In [3]:
learning_rate = 0.0001
buffer_size = 519
batch_size = 16
epochs = 500
img_size = 224
key = 12

In [4]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

training_set = train_datagen.flow_from_directory('output/train/',
                                                 target_size=(224,224),
                                                 batch_size=16)

test_set = test_datagen.flow_from_directory('output/val',
                                            target_size=(224,224),
                                            batch_size=16)

Found 2058 images belonging to 12 classes.
Found 545 images belonging to 12 classes.


In [5]:
def InceptionResNetV2_model():

    engine = tf.keras.applications.Xception(
        # Freezing the weights of the top layer in the InceptionResNetV2 pre-traiined model
        include_top = False,

        # Use Imagenet weights
        weights = 'imagenet',

        # Define input shape to 224x224x3
        input_shape = (img_size , img_size , 3),

    )

    x = tf.keras.layers.GlobalAveragePooling2D(name = 'avg_pool')(engine.output)
    x =Dropout(0.90)(x)
    out = tf.keras.layers.Dense(key, activation = 'softmax', name = 'dense_output')(x)


    # Build the Keras model

    model = tf.keras.models.Model(inputs = engine.input, outputs = out)
    # Compile the model

    model.compile(
        # Set optimizer to Adam(0.0001)
        optimizer = tf.keras.optimizers.Adam(learning_rate= 3e-4),
        #optimizer= SGD(lr=3e-4, momentum=0.9),
        # Set loss to binary crossentropy
        #loss = tf.keras.losses.sparse_categorical_crossentropy,
        loss = tf.keras.losses.CategoricalCrossentropy(),
        # Set metrics to accuracy
        metrics = ['accuracy']
    )

    return model

In [6]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                                            patience=2,
                                                            verbose=2,
                                                            factor=0.5,
                                                            min_lr=0.00001)
reduce_lr =  keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=0.00001)

In [7]:
def train():
    time_start = time.time()
    
    model = InceptionResNetV2_model()
    
    model.summary()
    history = model.fit(training_set, epochs= epochs , verbose = 2, validation_data = test_set,
                       callbacks=[early_stopping, reduce_lr , learning_rate_reduction])
    
    

    model.save_weights('layer22-wskindiseases1.h5')
    model.save('layer22-skindiseases1.h5')
    
    print('Model saved.')
    
    time_end = time.time()
    print('Training Time:', time_end - time_start)
    print('\n')

    return history


In [9]:
def test():
    #test_labels = np.array(test_labels)

    from tensorflow import keras
    print('Testing:')
    mod = InceptionResNetV2_model()
    mod.load_weights('Layer22-wskindiseases1.h5')
    mod.evaluate(test_set)
    
    #prob = mod.predict(test_set)
    #predIdxs = np.argmax(prob, axis=1) 


    print('\n')
    #print(classification_report(test_set.labels, predIdxs,target_names = key, digits=5))
    return predIdxs, prob, mod 

In [ ]:
if __name__ == "__main__":

    train_history = train() 
    predIdxs,prob,  model = test()
    
    show_train_history(train_history, 'sparse_categorical_accuracy')

In [10]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
input_path = 'Data/'
train_data = image_dataset_from_directory(directory=input_path,
                                          batch_size=16,
                                          image_size=(224, 224))
key = train_data.class_names

Found 2289 files belonging to 12 classes.


In [11]:
def read_directory():
    data_filenames = []
    data_labels = []
    for i in range (len(key)):
        for filename in os.listdir(input_path + key[i]):
                data_filenames.append(input_path +key[i]+'/' + filename)
                data_labels.append(i)
  
    return  data_filenames, data_labels

In [12]:
def build_test_tfrecord(train_filenames, train_labels):  # Generate TFRecord of training set 
    with tf.io.TFRecordWriter(test_tfrecord)as writer:
        for filename, label in zip(train_filenames, train_labels):
            image = open(filename, 'rb').read()

            feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),  # img > Bytes
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))  # label > Int
            }

            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())

In [13]:
def _parse_example(example_string):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }

    feature_dict = tf.io.parse_single_example(example_string, feature_description)
    feature_dict['image'] = tf.io.decode_png(feature_dict['image'], channels=3)
    feature_dict['image'] = tf.image.resize(feature_dict['image'], [img_size, img_size]) / 255.0
    return feature_dict['image'], feature_dict['label']

In [14]:
def get_test_dataset(test_tfrecord):
    raw_test_dataset = tf.data.TFRecordDataset(test_tfrecord)
    test_dataset = raw_test_dataset.map(_parse_example)

    return test_dataset

In [15]:
def data_Preprocessing( test_dataset):
    test_dataset = test_dataset.batch(batch_size)
    test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return  test_dataset

In [16]:
def InceptionResNetV2_model():

    engine = tf.keras.applications.Xception(
        # Freezing the weights of the top layer in the InceptionResNetV2 pre-traiined model
        include_top = False,

        # Use Imagenet weights
        weights = 'imagenet',

        # Define input shape to 224x224x3
        input_shape = (img_size , img_size , 3),

    )

    x = tf.keras.layers.GlobalAveragePooling2D(name = 'avg_pool')(engine.output)
    x =Dropout(0.85)(x)
    out = tf.keras.layers.Dense(len(key), activation = 'softmax', name = 'dense_output')(x)


    # Build the Keras model

    model = tf.keras.models.Model(inputs = engine.input, outputs = out)
    # Compile the model

    model.compile(
        # Set optimizer to Adam(0.0001)
        optimizer = tf.keras.optimizers.Adam(learning_rate= 3e-4),
        #optimizer= SGD(lr=0.0001, momentum=0.9),
        # Set loss to binary crossentropy
        loss = tf.keras.losses.sparse_categorical_crossentropy,

        # Set metrics to accuracy
        metrics = ['accuracy']
    )


    return model

In [17]:
def test(test_labels):
    test_labels = np.array(test_labels)

    from tensorflow import keras
    print('Testing:')
    mod = InceptionResNetV2_model()
    mod.load_weights('final version (94)/layer22-wskindiseases1.h5')
    mod.evaluate(test_dataset)
    
    prob = mod.predict(test_dataset)
    predIdxs = np.argmax(prob, axis=1) 


    print('\n')
    print(classification_report(test_labels, predIdxs,target_names = key, digits=5))
    return test_labels, predIdxs, prob, mod

In [18]:
if __name__ == "__main__":
    
    test_filenames, test_labels = read_directory()
    test_tfrecord = 'XRay_test.tfrecords'
    build_test_tfrecord(test_filenames, test_labels)


    test_dataset = get_test_dataset(test_tfrecord)



    test_dataset = data_Preprocessing(test_dataset) 

    test_labels, predIdxs,prob, Model = test(test_labels)

Testing:
144/144 [==============================] - 17s 97ms/step - loss: 0.0748 - accuracy: 0.9803.0788 - ac


                                              precision    recall  f1-score   support

                 Acquired ichthyosis - Adult    0.96774   0.96774   0.96774        62
               Atopic Dermatitis Adult Phase    0.98834   0.97696   0.98262       434
           Atopic Dermatitis Childhood Phase    0.95489   0.98961   0.97194       385
                      Atopic Dermatitis Feet    0.97076   0.95954   0.96512       173
       Atopic Dermatitis Hyperlinear Creases    0.97778   0.98876   0.98324        89
                Harlequin ichthyosis - Child    1.00000   0.96875   0.98413        32
                         Ichthyosis vulgaris    1.00000   0.97619   0.98795        84
                           Keratosis Pilaris    0.99814   0.98175   0.98988       548
Keratosis pilaris atrophicans faciei - Adult    1.00000   0.90909   0.95238        22
                 Lamellar i

In [19]:
def dermai_accuracy(prob, prob_Desicon):
    dermai = []
    for i in range(len(prob)):
        catogery =[]
        for j in range(len(prob[i])):
            if prob[i][j] > prob_Desicon:
                catogery.append(j)
        dermai.append(catogery)
    Positive = []
    for i in range(len(dermai)):
        if test_labels[i] in dermai[i]:
            Positive.append(1)
    accuracy = (len(Positive)/len(test_labels))*100  
    return  accuracy,dermai

In [20]:
accuracy,dermai = dermai_accuracy(prob, 0.10)

In [ ]:
accuracy